<a href="https://colab.research.google.com/github/tmskss/Flan-T5-K8S-QA/blob/main/notebooks/evaluate_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is used to evaluate models with ROUGE metrics using a specified dataset

In [1]:
!pip install transformers[torch] sentencepiece tokenizers evaluate datasets rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-

In [2]:
from datasets import load_dataset

dataset = load_dataset('ComponentSoft/k8s-kubectl')['validate']

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3161 [00:00<?, ? examples/s]

Generating validate split:   0%|          | 0/316 [00:00<?, ? examples/s]

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained('ComponentSoft/flan-t5-base-k8s-question-code')
model = AutoModelForSeq2SeqLM.from_pretrained('ComponentSoft/flan-t5-base-k8s-question-code').to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import evaluate

rouge = evaluate.load('rouge')
rouge_scores = []
outputs = []

In [5]:
for item in dataset:
  # Tokenize text
  input_ids = tokenizer.encode(item['question'], return_tensors="pt", max_length=512, truncation=True).to('cuda')

  # Generate output
  output_ids = model.generate(input_ids, max_length=256, num_return_sequences=1, num_beams=4)

  # Decode output
  generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

  # Create rouge metrics
  predictions = [generated_text]
  references = [item['code']]
  results = rouge.compute(predictions=predictions, references=references)

  rouge_scores.append(results)

  # Print results
  print('Question: ' + item['question'] + '\nGenerated output: ' + generated_text + '\nReference: ' + item['code'] + '\nROUGE scores: ' + str(results) + '\n--------------------------------------------------------------------------')
  outputs.append('Question: ' + item['question'] + '\nGenerated output: ' + generated_text + '\nReference: ' + item['code'] + '\nROUGE scores: ' + str(results) + '\n--------------------------------------------------------------------------')

Question: Create a pod using the data in pod.json
Generated output: kubectl create -f./pod.json
Reference: kubectl create -f ./pod.json
ROUGE scores: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
--------------------------------------------------------------------------
Question: Create a pod based on the JSON passed into stdin
Generated output: cat pod.json | kubectl create -f -
Reference: cat pod.json | kubectl create -f -
ROUGE scores: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
--------------------------------------------------------------------------
Question: Edit the data in docker-registry.yaml in JSON then create the resource using the edited data
Generated output: kubectl create -f docker-registry.yaml --edit -o json
Reference: kubectl create -f docker-registry.yaml --edit -o json
ROUGE scores: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}
--------------------------------------------------------------------------
Questio

In [6]:
# Calculate average from rouge scores
avg_scores = {
    'avg_rouge1': sum(score['rouge1'] for score in rouge_scores) / len(rouge_scores),
    'avg_rouge2': sum(score['rouge2'] for score in rouge_scores) / len(rouge_scores),
    'avg_rougeL': sum(score['rougeL'] for score in rouge_scores) / len(rouge_scores),
    'avg_rougeLsum': sum(score['rougeLsum'] for score in rouge_scores) / len(rouge_scores),
}

print('Average ROUGE scores: ' + str(avg_scores))

Average ROUGE scores: {'avg_rouge1': 0.964986437780824, 'avg_rouge2': 0.9513970308309867, 'avg_rougeL': 0.9636096500116312, 'avg_rougeLsum': 0.9636096500116312}


In [7]:
# Dump output to a txt file
file_name = 'output.txt'

with open(file_name, 'w') as file:
    for item in outputs:
        file.write(f"{item}\n")

    file.write('Average ROUGE scores: ' + str(avg_scores))